# Introduction, imports, and function defs

Preprocessing steps needed for Goodreads books (Mystery/Crime/Thriller subset) and authors datasets. Data retrieved from: https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home?authuser=0. Specifically, in this notebook for the sake of exploration, I will be cleaning the mysteries/thriller genre subset of books. The full authors list will be used.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gdown
import seaborn as sns
from google.colab import drive
import json
from os.path import join
import re
import random

In [2]:
# Mount drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [3]:
dir = '/content/drive/Shareddrives/CIS5500-BookReviews'
#authors_path = 'goodreads_book_authors.json'
books_path = 'RawFiles/goodreads_books_mystery_thriller_crime.json'

In [14]:
# Function to load the data from the unzipped json files:
def load_json(file_name, stop = 500):
  ''' file_name: path to the json file to be loaded
      head: the number of elements to load
  '''
  count = 0
  data = []
  with open(file_name) as fn:
      for l in fn:
          d = json.loads(l)
          count += 1
          data.append(d)
          
          if (stop is not None) and (count > stop):
              break
  return data

# Authors data loading and cleaning

In [ ]:
authors_data = load_json(join(dir, authors_path), head=None)

In [ ]:
authors_df = pd.DataFrame(authors_data)
authors_df.head()

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [ ]:
authors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829529 entries, 0 to 829528
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   average_rating      829529 non-null  object
 1   author_id           829529 non-null  object
 2   text_reviews_count  829529 non-null  object
 3   name                829529 non-null  object
 4   ratings_count       829529 non-null  object
dtypes: object(5)
memory usage: 31.6+ MB


In [ ]:
authors_df = authors_df[['author_id', 'average_rating', 'text_reviews_count', 'name', 'ratings_count']]
authors_df.head()

,author_id,average_rating,text_reviews_count,name,ratings_count
0,604031,3.98,7,Ronald J. Fields,49
1,626222,4.08,28716,Anita Diamant,546796
2,10333,3.92,5075,Barbara Hambly,122118
3,9212,3.68,36262,Jennifer Weiner,888522
4,149918,3.82,96,Nigel Pennick,1740


In [ ]:
authors_df.tail()

,author_id,average_rating,text_reviews_count,name,ratings_count
829524,197551,4.36,4,Patty Furbush,11
829525,3988103,4.33,3,Jim Schlinkman,6
829526,13464507,4.00,2,Rich Jolly,18
829527,7427847,3.31,1,sr@ mwrGn,13
829528,5401342,3.70,11,Barry S. Brown,43


In [ ]:
# Ensure that there are no missing values in this table
assert(authors_df[authors_df.isna().any(axis=1)].shape == (0, len(authors_df.columns)))

In [ ]:
# Ensure that each author has a unique ID
assert(authors_df['author_id'].nunique() == authors_df.shape[0])

In [ ]:
authors_df[authors_df['name'].str.contains(',')]

,author_id,average_rating,text_reviews_count,name,ratings_count
1329,4599042,4.31,19,"Yudis, Broky, Pak Waw",131
1381,6936990,5.00,1,"Jiao Chuan gemusu, gurasuhotsupamanihuakuchiyua",2
1674,7325512,4.09,55,"Madden, Colleen M.",392
3877,4878973,4.75,3,"Bill Kovach, Tom Rosenstiel",8
3937,2278579,2.11,4,"Meryl Dory, Susan Lindberg, Stephanie Messenger",9
...,...,...,...,...,...
829199,296279,3.90,8,"Esther Allen, trans.",118
829204,8056403,3.40,3,"Charles Corey, C.B. Corey",10
829205,8056404,3.40,3,"Hesketh Prichard, Kate Prichard, E. Heron, H. ...",10
829258,7172301,2.33,1,"Wahib Saray al-Din ,whyb sry ldyn",3


In [ ]:
3177 / 829528

0.0038298888042356616

In [ ]:
split_names = authors_df['name'].str.split(',')

In [ ]:
def FindMaxLength(lst):
    maxList = max(lst, key=len)
    maxLength = len(maxList)
     
    return maxList, maxLength

In [ ]:
FindMaxLength(split_names)

(['Saroyan',
  ' Cunninghan',
  ' Mazilu',
  ' Pantu',
  ' Sibisteanu',
  ' Vieru',
  ' Grosan',
  ' Ilea',
  ' Popa',
  ' Pricajan',
  ' Romila',
  ' Lungu'],
 12)

In [ ]:
# Possible TODO: split multi-author lists into multiple rows

The authors data is clean and ready for loading into the Authors Table in our database

# Books data loading and cleaning (Mysteries/Thrillers/Crime genre subset)

In [15]:
books_data = load_json(join(dir, books_path), stop=None)

In [16]:
books_raw = pd.DataFrame(books_data)
books_raw.head(10)

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,184737297X,15,[169353],US,,"[{'count': '159', 'name': 'to-read'}, {'count'...",,false,3.93,B007YLTG5I,...,4,,2009,https://www.goodreads.com/book/show/6066814-cr...,https://images.gr-assets.com/books/1328724803m...,6066814,186,6243149,"Crowner Royal (Crowner John Mystery, #13)","Crowner Royal (Crowner John Mystery, #13)"
1,,60,[1052227],US,eng,"[{'count': '54', 'name': 'currently-reading'},...",B01NCIKAQX,true,4.33,B01NCIKAQX,...,,,,https://www.goodreads.com/book/show/33394837-t...,https://images.gr-assets.com/books/1493114742m...,33394837,269,54143148,The House of Memory (Pluto's Snitch #2),The House of Memory (Pluto's Snitch #2)
2,,23,[953679],US,eng,"[{'count': '90', 'name': 'to-read'}, {'count':...",B01ALOWJN0,true,3.49,B01ALOWJN0,...,,,,https://www.goodreads.com/book/show/29074697-t...,https://s.gr-assets.com/assets/nophoto/book/11...,29074697,192,49305010,The Slaughtered Virgin of Zenopolis (Inspector...,The Slaughtered Virgin of Zenopolis (Inspector...
3,0854563903,8,[408775],US,,"[{'count': '51', 'name': 'to-read'}, {'count':...",,false,3.30,,...,12,Large Print,1975,https://www.goodreads.com/book/show/1902202.De...,https://s.gr-assets.com/assets/nophoto/book/11...,1902202,52,1903897,"Dead in the Morning (Patrick Grant, #1)","Dead in the Morning (Patrick Grant, #1)"
4,8838920931,3,[274410],US,ita,"[{'count': '48', 'name': 'to-read'}, {'count':...",,false,3.54,,...,,,2006,https://www.goodreads.com/book/show/9671977-ar...,https://images.gr-assets.com/books/1474788304m...,9671977,22,2152906,Aristotele e i misteri di Eleusi,Aristotele e i misteri di Eleusi
5,0062265806,3,[199039],US,eng,"[{'count': '1694', 'name': 'mystery'}, {'count...",,false,3.96,,...,3,,2013,https://www.goodreads.com/book/show/16158998-a...,https://images.gr-assets.com/books/1360572193m...,16158998,5,2288775,A Murder is Announced,A Murder is Announced
6,,5,[],US,en-GB,"[{'count': '27', 'name': 'to-read'}, {'count':...",B00UQVGQMO,true,3.80,B00UQVGQMO,...,,,,https://www.goodreads.com/book/show/25162836-d...,https://s.gr-assets.com/assets/nophoto/book/11...,25162836,8,44866515,Dark Flames Rising,Dark Flames Rising
7,0752844458,8,[326237],US,,"[{'count': '38', 'name': 'to-read'}, {'count':...",,false,3.61,B00KKFTAL0,...,8,,2001,https://www.goodreads.com/book/show/2805495-wy...,https://images.gr-assets.com/books/1328819096m...,2805495,58,2831381,Wycliffe and the Cycle of Death,Wycliffe and the Cycle of Death
8,8293326247,6,[],US,eng,"[{'count': '171', 'name': 'to-read'}, {'count'...",,false,4.14,,...,11,,2014,https://www.goodreads.com/book/show/22722787-t...,https://s.gr-assets.com/assets/nophoto/book/11...,22722787,18,42251489,The Cost of Doing Business,The Cost of Doing Business
9,0062265784,2,[199041],US,eng,"[{'count': '1642', 'name': 'mystery'}, {'count...",,false,3.93,,...,3,,2013,https://www.goodreads.com/book/show/16158996-4,https://images.gr-assets.com/books/1360566349m...,16158996,5,6490729,4:50 From Paddington,4:50 From Paddington


In [17]:
keep_cols = ['isbn', 'text_reviews_count', 'series', 'language_code', 'is_ebook', 'average_rating', 'similar_books',
             'description', 'format', 'authors', 'publisher', 'num_pages', 'publication_year', 'image_url',
             'book_id', 'ratings_count', 'title']

TODOS:


*   Replace missing values: may need to have column-specific solutions to this ie. 'Not Available' string for ISBN or a url to a 'missing' image for not available book icons
*   remove brackets from 'series' column numbers
*   Replace value in 'series' for books that aren't part of one with a coded value (-999 maybe?) so that code can be written to check for this value when rendering book page.
*   Extract the 'similar_books' list and keep up to three similar books. Create another separate table of book IDs and retained similar book IDs.
*   Check book IDs are unique
*   cast year as datetime year object
*   Extract authors list and split into separate table, WrittenBy(book_id, author_id, role)
*   Remove any book with no listed authors
 











In [18]:
books_raw = books_raw[keep_cols]

## Process Similar Books Table 
1.   Consists of columns 'similar_books' and 'book_id'
2.   Books with no similar books will be assigned a NULL value in the similar books column. 
3.   Up to 3 similar books will be extracted from the lists of book_ids with similar books



In [9]:
# SimilarBooks(book_id, similar_book_id)
similar_books_raw = books_raw[['similar_books', 'book_id']]

In [10]:
similar_books_raw

,similar_books,book_id
0,"[439108, 522621, 116770, 1275927, 6202655, 840...",6066814
1,[],33394837
2,[],29074697
3,[],1902202
4,[],9671977
...,...,...
219230,"[6172075, 21835, 4583828, 2797862, 237181, 232...",9877072
219231,"[23012660, 13763625, 18079894, 21432940, 17343...",26591045
219232,"[24808217, 17620517, 13496536, 21947925, 18079...",25727303
219233,[],23252156


In [30]:
def clean_similar_books(x, keep_count = 3):

  # Row with a list of similar books
  inds = x['similar_books'].astype('bool')

  # Extract non-empty rows
  x_similar = x[inds].copy()
  x_similar = x_similar.explode('similar_books')
  # Remove the values that are not in book_id set
  x_similar = x_similar[x_similar['similar_books'].isin(x_similar['book_id'])]

  cleaned = x_similar.groupby('book_id').head(keep_count).reset_index(drop=True)

  return cleaned[['book_id', 'similar_books']]


In [31]:
num_similar_to_keep = 3
similar_books_clean = clean_similar_books(similar_books_raw, keep_count=num_similar_to_keep)
similar_books_clean.head()

,book_id,similar_books
0,6066814,439108
1,6066814,522621
2,6066814,116770
3,16158998,93575
4,16158998,280849


In [32]:
# Sanity chack that we have the correct max number of similar books per book_id
assert(similar_books_clean.groupby('book_id')['similar_books'].count().max() == num_similar_to_keep)

In [33]:
# Sanity check that all similar book ids are in also in Books(book_id) column
assert(all(similar_books_clean['similar_books'].isin(similar_books_raw['book_id'])))

In [34]:
similar_books_clean.shape

(443622, 2)

In [35]:
similar_books_clean.to_csv(join(dir, 'CleanedFilesForDB/similar_books.csv'), header=True, index=False)

In [ ]:
del similar_books_raw, similar_books_clean

## Process WrittenBy Table 
1.   Consists of columns 'author_id', 'book_id', and 'author_role'
2.   Books with no attributed author will be assigned a NULL value in the author_id and role columns. 

In [19]:
# WrittenBy(book_id, author_id, author_role) 
written_by_raw = books_raw[['authors', 'book_id']]


In [20]:
written_by_raw.head()

,authors,book_id
0,"[{'author_id': '37778', 'role': ''}]",6066814
1,"[{'author_id': '242185', 'role': ''}]",33394837
2,"[{'author_id': '15104629', 'role': ''}, {'auth...",29074697
3,"[{'author_id': '190988', 'role': ''}]",1902202
4,"[{'author_id': '337108', 'role': ''}, {'author...",9671977


In [21]:
def clean_written_by(x):
  tmp = x.explode('authors').copy()
  aut_role_series = tmp['authors'].apply(lambda x : x.values() )
  tmp[['author_id', 'role']] = pd.DataFrame(aut_role_series.to_list(), index=tmp.index)
  tmp.drop('authors', axis=1, inplace=True)
  #tmp[['author_id', 'book_id']] = tmp[['author_id', 'book_id']].astype('float64')

  return tmp

In [22]:
written_cleaned = clean_written_by(written_by_raw)

In [23]:
written_cleaned.shape

(283874, 3)

In [24]:
written_cleaned.drop_duplicates(subset=['book_id', 'author_id'], inplace=True)

In [25]:
written_cleaned.shape

(283692, 3)

In [ ]:
written_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283874 entries, 0 to 219234
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   book_id    283874 non-null  object
 1   author_id  283874 non-null  object
 2   role       283874 non-null  object
dtypes: object(3)
memory usage: 8.7+ MB


In [ ]:
# Find maximum character length in the role column in ordder to define VARCHAR
max(written_cleaned.role.astype('str').apply(len))

32

In [26]:
written_cleaned.to_csv(join(dir, 'CleanedFilesForDB/written_by.csv'), header=True, index=False)

In [ ]:
del written_cleaned

## Process Books Table 
1.   Drop the similar_books and authors columns as these are in the separate relations above.
2.   Missing data kept as empty string [ should be read in to SQL DB as NULL ]. Exception for numeric columns in which it makes sense to encode 0 for missing data.
3. 

In [ ]:
# Books(keep_cols drop similar_books and authors)
books_raw = books_raw.drop(['similar_books', 'authors'], axis=1)

In [ ]:
books_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219235 entries, 0 to 219234
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   isbn                219235 non-null  object
 1   text_reviews_count  219235 non-null  object
 2   series              219235 non-null  object
 3   language_code       219235 non-null  object
 4   is_ebook            219235 non-null  object
 5   average_rating      219235 non-null  object
 6   description         219235 non-null  object
 7   format              219235 non-null  object
 8   publisher           219235 non-null  object
 9   num_pages           219235 non-null  object
 10  publication_year    219235 non-null  object
 11  image_url           219235 non-null  object
 12  book_id             219235 non-null  object
 13  ratings_count       219235 non-null  object
 14  title               219235 non-null  object
dtypes: object(15)
memory usage: 25.1+ MB


In [ ]:
null_replacements = {'text_reviews_count' : 0,
                     'average_rating' : 0.00,
                     'ratings_count' : 0,
                     }


def clean_books_table(df, replace_dict):
  x = df.copy()
  # Replace empty strings in [isbn, language_code] columns with NaN
  x.replace(to_replace='', value=replace_dict, inplace=True)
  
  # # Convert text_review_counts to numbers
  # # Check that there are no misssing values
  # if (x[x['text_reviews_count'] == ''].shape[0] == 0):
  #   x['text_reviews_count'] = x['text_reviews_count'].astype('int64')

  # # Convert average_rating to numbers
  # # Check that there are no misssing values
  # if (x[x['average_rating'] == ''].shape[0] == 0):
  #   x['average_rating'] = x['average_rating'].astype('float64')

  # # Convert num_pages to numbers
  # # Check that there are no misssing values
  # if (x[x['num_pages'] == ''].shape[0] == 0):
  #   x['num_pages'] = x['num_pages'].astype('float64')

  # x['book_id'] = x['book_id'].astype('int64')
    
  # Allow a series to be part of at most one series
  x.loc[:, 'series'] = x.loc[:,'series'].apply(lambda y : y[0] if y else '')
  x['series'] = x['series'].astype('str')

  x.loc[:, 'is_ebook'] = x.loc[:,'is_ebook'].apply(lambda y : 1 if y == 'true' else 0)

  #cols_to_int = ['is_ebook', 'text_reviews_count', 'average_rating', 'num_pages', 'book_id', 'series']
  #x[cols_to_int] = x[cols_to_int].astype('float64')

  return x
  
  

In [ ]:
books_cleaned = clean_books_table(books_raw, null_replacements)

In [ ]:
books_cleaned.head()

,isbn,text_reviews_count,series,language_code,is_ebook,average_rating,description,format,publisher,num_pages,publication_year,image_url,book_id,ratings_count,title
0,184737297X,15,169353,,0,3.93,"London, 1196. At the command of Richard the Li...",Hardcover,Simon & Schuster UK,400,2009,https://images.gr-assets.com/books/1328724803m...,6066814,186,"Crowner Royal (Crowner John Mystery, #13)"
1,,60,1052227,eng,1,4.33,,,,318,,https://images.gr-assets.com/books/1493114742m...,33394837,269,The House of Memory (Pluto's Snitch #2)
2,,23,953679,eng,1,3.49,"BATHS, BANKS AND ROMAN INSURRECTION\nDetective...",,Amazon Digital Services,,,https://s.gr-assets.com/assets/nophoto/book/11...,29074697,192,The Slaughtered Virgin of Zenopolis (Inspector...
3,0854563903,8,408775,,0,3.30,"Gerald breezily introduced his wife, Helen, to...",Hardcover,Ulverscroft,,1975,https://s.gr-assets.com/assets/nophoto/book/11...,1902202,52,"Dead in the Morning (Patrick Grant, #1)"
4,8838920931,3,274410,ita,0,3.54,"""I misteri di Eleusi"" e il quinto romanzo di A...",Paperback,Sellerio,659,2006,https://images.gr-assets.com/books/1474788304m...,9671977,22,Aristotele e i misteri di Eleusi


In [ ]:
books_cleaned.to_csv(join(dir, 'CleanedFilesForDB/books.csv'), header=True)

# A few extra cleaning steps for various tables

## Books table: Change column order and find max character numbers for a few columns (for VARCHAR)

In [ ]:
data = pd.read_csv(join(dir, 'CleanedFilesForDB/books.csv'))

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219235 entries, 0 to 219234
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          219235 non-null  int64  
 1   isbn                134718 non-null  object 
 2   text_reviews_count  219235 non-null  int64  
 3   series              131828 non-null  float64
 4   language_code       136784 non-null  object 
 5   is_ebook            219235 non-null  int64  
 6   average_rating      219235 non-null  float64
 7   description         198488 non-null  object 
 8   format              162390 non-null  object 
 9   publisher           161303 non-null  object 
 10  num_pages           149407 non-null  float64
 11  publication_year    164741 non-null  float64
 12  image_url           219235 non-null  object 
 13  book_id             219235 non-null  int64  
 14  ratings_count       219235 non-null  int64  
 15  title               219235 non-nul

In [ ]:
max(data.isbn.map(str).apply(len))

10

In [ ]:
import numpy as np


max(data.title.astype(str).apply(len))


245

In [ ]:
data = data[['isbn', 'book_id', 'text_reviews_count', 'series', 'language_code', 'is_ebook', 'average_rating', 'description', 'format', 'publisher', 'num_pages', 'publication_year', 'image_url', 'ratings_count', 'title']]

In [ ]:
data.columns

Index(['isbn', 'book_id', 'text_reviews_count', 'series', 'language_code',
       'is_ebook', 'average_rating', 'description', 'format', 'publisher',
       'num_pages', 'publication_year', 'image_url', 'ratings_count', 'title'],
      dtype='object')

In [ ]:
data.rename(columns={'series': 'series_id'}, inplace=True)

<ipython-input-17-257a91917c3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'series': 'series_id'}, inplace=True)


In [ ]:
data.head()

,isbn,book_id,text_reviews_count,series_id,language_code,is_ebook,average_rating,description,format,publisher,num_pages,publication_year,image_url,ratings_count,title
0,184737297X,6066814,15,169353.0,NaN,0,3.93,"London, 1196. At the command of Richard the Li...",Hardcover,Simon & Schuster UK,400.0,2009.0,https://images.gr-assets.com/books/1328724803m...,186,"Crowner Royal (Crowner John Mystery, #13)"
1,NaN,33394837,60,1052227.0,eng,1,4.33,NaN,NaN,NaN,318.0,NaN,https://images.gr-assets.com/books/1493114742m...,269,The House of Memory (Pluto's Snitch #2)
2,NaN,29074697,23,953679.0,eng,1,3.49,"BATHS, BANKS AND ROMAN INSURRECTION\nDetective...",NaN,Amazon Digital Services,NaN,NaN,https://s.gr-assets.com/assets/nophoto/book/11...,192,The Slaughtered Virgin of Zenopolis (Inspector...
3,0854563903,1902202,8,408775.0,NaN,0,3.30,"Gerald breezily introduced his wife, Helen, to...",Hardcover,Ulverscroft,NaN,1975.0,https://s.gr-assets.com/assets/nophoto/book/11...,52,"Dead in the Morning (Patrick Grant, #1)"
4,8838920931,9671977,3,274410.0,ita,0,3.54,"""I misteri di Eleusi"" e il quinto romanzo di A...",Paperback,Sellerio,659.0,2006.0,https://images.gr-assets.com/books/1474788304m...,22,Aristotele e i misteri di Eleusi


In [ ]:
data.to_csv(join(dir, 'CleanedFilesForDB/books.csv'), header=True, index=False)

In [ ]:
data.tail()

,isbn,book_id,text_reviews_count,series_id,language_code,is_ebook,average_rating,description,format,publisher,num_pages,publication_year,image_url,ratings_count,title
219230,6054188089,9877072,1,NaN,tur,0,3.86,"""Sayfalari cevirirken nefesinizin kesildigini ...",Paperback,Koridor Yayincilik,410.0,2009.0,https://images.gr-assets.com/books/1327590639m...,37,Deja Vu
219231,1780928157,26591045,6,NaN,NaN,0,3.81,From the author of The Cotswolds Werewolf come...,Paperback,MX Publishing,208.0,2015.0,https://images.gr-assets.com/books/1444637176m...,16,The Sensible Necktie and Other Stories of Sher...
219232,1780927886,25727303,3,NaN,eng,0,4.11,"Accompanied by Holmes and Watson, Mrs. Hudson ...",Paperback,MX Publishing,288.0,2015.0,https://images.gr-assets.com/books/1434310664m...,6,Mrs. Hudson in New York
219233,NaN,23252156,7,NaN,eng,1,4.00,"When Kirk Stanford moves into his new home, st...",ebook,NaN,NaN,2014.0,https://images.gr-assets.com/books/1411156827m...,16,Wicked Reflection
219234,178092870X,26168430,2,NaN,eng,0,3.50,Sir Arthur Conan Doyle is brought back to life...,Paperback,MX Publishing,148.0,2015.0,https://images.gr-assets.com/books/1440592011m...,6,Sherlock Holmes and the July Crisis


In [ ]:
data[data['book_id'] == '']

,isbn,book_id,text_reviews_count,series_id,language_code,is_ebook,average_rating,description,format,publisher,num_pages,publication_year,image_url,ratings_count,title


In [ ]:
!unzip /content/drive/Shareddrives/CIS5500-BookReviews/RawFiles/goodreads_book_genres.json.gz

Archive:  /content/drive/Shareddrives/CIS5500-BookReviews/RawFiles/goodreads_book_genres.json.gz
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/Shareddrives/CIS5500-BookReviews/RawFiles/goodreads_book_genres.json.gz or
        /content/drive/Shareddrives/CIS5500-BookReviews/RawFiles/goodreads_book_genres.json.gz.zip, and cannot find /content/drive/Shareddrives/CIS5500-BookReviews/RawFiles/goodreads_book_genres.json.gz.ZIP, period.


## Check genres table

In [ ]:
genre_path = 'RawFiles/goodreads_book_genres.json'

In [ ]:
genres = load_json(join(dir, genre_path), stop=None)


AttributeError: ignored

In [ ]:
genres = pd.DataFrame(genres)

In [ ]:
genres.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [ ]:
genres_exploded = genres.explode('genres')

In [ ]:
genres_exploded.value_counts('genres')

genres
fiction                                   1244112
history, historical fiction, biography     663795
romance                                    658719
fantasy, paranormal                        538311
non-fiction                                533491
mystery, thriller, crime                   523156
young-adult                                364114
children                                   256935
comics, graphic                            171279
poetry                                      88630
dtype: int64

## Checking Authors

In [37]:
authors = pd.read_csv(join(dir, 'CleanedFilesForDB/authors.csv'))

In [38]:
authors.head()

,author_id,name,average_rating,text_reviews_count,ratings_count
0,604031,Ronald J. Fields,3.98,7,49
1,626222,Anita Diamant,4.08,28716,546796
2,10333,Barbara Hambly,3.92,5075,122118
3,9212,Jennifer Weiner,3.68,36262,888522
4,149918,Nigel Pennick,3.82,96,1740


In [39]:
# ensure there are no missing author ids
assert(authors[authors['author_id'] == ''].shape[0] == 0)

In [40]:
# 
max(authors.name.astype('str').apply(len))

116

In [41]:
authors.shape

(829529, 5)

## Remove any book ids that are not in the mysteries subset 

In [44]:
books_raw['book_id'].to_csv(join(dir, 'CleanedFilesForDB/book_id_set.csv'), header=True, index=False)

In [4]:
bg = pd.read_csv('/content/drive/Shareddrives/CIS5500-BookReviews/CleanedFilesForDB/books_genres.csv')

In [5]:
bg.head()

,book_id,genre_id,n_votes
0,5333265,1,1
1,5333265,2,1
2,5333265,3,1
3,1333909,4,219
4,1333909,1,5


In [6]:
bg.shape

(7459977, 3)

In [7]:
book_ids = pd.read_csv(join(dir, 'CleanedFilesForDB/book_id_set.csv'))


In [8]:
book_ids

,book_id
0,6066814
1,33394837
2,29074697
3,1902202
4,9671977
...,...
219230,9877072
219231,26591045
219232,25727303
219233,23252156


In [9]:
bg_myssteries = bg[bg['book_id'].isin(book_ids['book_id'])]

In [10]:
bg_myssteries.shape

(1128293, 3)

In [11]:
bg.shape[0] - bg_myssteries.shape[0]

6331684

In [12]:
bg_myssteries.to_csv(join(dir, 'CleanedFilesForDB/books_genres.csv'), header=True, index=False)